In [1]:
import pandas as pd 
import numpy as np
import glob
import os
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive

Informal ReadMe: 
- My top priority is not scalabilty (yet) 
- 

In [2]:
# Michael's Advice: 
    # One df per patient
    # Pivot? Get dummies?
    # Rolling Window   !!! Ben said 3 second window is best
    
# JM's Advice: 
    # Write a function to bring in all the data
    # Have it make a patient column and doc column
    # But getting an MVP could be super useful too. 
        # So if you want to try the experiment on pt 1, that's good too. 

# Experimentation with patient #1

CPG #1

In [3]:
CPG1 = pd.read_csv('Barretts_Analysis/CPG/001 CPG.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = CPG1.iloc[0]
header = header.str.replace('\n', ' ')
CPG1 = CPG1.rename(columns = header)
CPG1 = CPG1[1:]

CPG1 = CPG1.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

CPG1 = CPG1[CPG1["Meas Type"].str.contains("MII")]
CPG1 = CPG1.rename(columns={'Meas Start Time': 'CPG Start Time'})

In [4]:
Auto1 = pd.read_csv('Barretts_Analysis/Autoscan/001 Autoscan.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = Auto1.iloc[0]
header = header.str.replace('\n', ' ')
Auto1 = Auto1.rename(columns = header)
Auto1 = Auto1[1:]

Auto1 = Auto1.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

Auto1 = Auto1[Auto1["Meas Type"].str.contains("MII")]
Auto1 = Auto1.rename(columns={'Meas Start Time': 'Auto Start Time'})

In [5]:
CPG1

,Meas,CPG Start Time,Meas Type,Body Position,Contents,MII Reflux,pH Classification,
1,Meas 1:,1/16:42:24.0,MII,Upright,Mixed,Reflux,Acid,
12,Meas 4:,1/16:46:55.6,MII,Upright,Mixed,Reflux,Acid,
21,Meas 6:,1/16:57:22.0,MII,Upright,Mixed,Reflux,Acid,
30,Meas 8:,1/18:08:27.0,MII,Upright,Mixed,Reflux,NonAcid,
36,Meas 9:,1/18:32:44.0,MII,Upright,Mixed,Reflux,Acid,
46,Meas 11:,1/18:41:27.5,MII,Upright,Mixed,Reflux,Acid,
56,Meas 13:,1/18:53:01.0,MII,Upright,Mixed,Reflux,Acid,
65,Meas 15:,1/19:04:10.0,MII,Upright,Mixed,Reflux,Acid,
74,Meas 17:,1/19:05:54.1,MII,Upright,Mixed,Reflux,Acid,
84,Meas 19:,1/19:06:21.7,MII,Upright,Mixed,Reflux,Acid,


In [6]:
Auto1

,Meas,Auto Start Time,Meas Type,Body Position,Contents,MII Reflux,pH Classification,
1,Meas 1:,1/16:35:31.0,MII,Upright,Liquid,Reflux,NonAcid,
7,Meas 2:,1/16:42:24.0,MII,Upright,Mixed,Reflux,Acid,
18,Meas 5:,1/16:47:10.0,MII,Upright,Mixed,Reflux,Acid,
26,Meas 7:,1/16:57:22.0,MII,Upright,Mixed,Reflux,Acid,
35,Meas 9:,1/18:08:27.0,MII,Upright,Mixed,Reflux,NonAcid,
41,Meas 10:,1/18:32:44.0,MII,Upright,Mixed,Reflux,Acid,
51,Meas 12:,1/18:41:27.5,MII,Upright,Mixed,Reflux,Acid,
61,Meas 14:,1/18:53:01.0,MII,Upright,Mixed,Reflux,Acid,
71,Meas 16:,1/19:04:10.0,MII,Upright,Mixed,Reflux,Acid,
80,Meas 18:,1/19:05:54.1,MII,Upright,Mixed,Reflux,Acid,


In [7]:
Auto1.shape

(39, 8)

# Test 1: merge

# Test 2: Concatenating CPG + Auto - Patient 1

In [8]:
TestCPGAuto2 = pd.concat([CPG1, Auto1], axis = 1).fillna('').drop(columns = {'Meas Type', 'Body Position', 'Contents'})

In [9]:
TestCPGAuto2

,Meas,CPG Start Time,MII Reflux,pH Classification,,Meas,Auto Start Time,MII Reflux,pH Classification,
1,Meas 1:,1/16:42:24.0,Reflux,Acid,,Meas 1:,1/16:35:31.0,Reflux,NonAcid,
7,,,,,,Meas 2:,1/16:42:24.0,Reflux,Acid,
12,Meas 4:,1/16:46:55.6,Reflux,Acid,,,,,,
18,,,,,,Meas 5:,1/16:47:10.0,Reflux,Acid,
21,Meas 6:,1/16:57:22.0,Reflux,Acid,,,,,,
...,...,...,...,...,...,...,...,...,...,...
341,,,,,,Meas 83:,2/12:08:02.6,No Reflux,Needs pH,
348,Meas 86:,2/12:33:02.9,Reflux,Acid,,,,,,
349,,,,,,Meas 86:,2/12:18:02.0,Reflux,Acid,
365,,,,,,Meas 91:,2/12:32:51.0,Reflux,Acid,


In [10]:
TestCPGAuto2.shape

(63, 10)

In [11]:
# If you have a Doc column again and stack vertically, you could sort by start time. 
# Probably need Start Day and Start Time re.split

In [12]:
TestCPGAuto2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 1 to 377
Data columns (total 10 columns):
Meas                 63 non-null object
CPG Start Time       63 non-null object
MII Reflux           63 non-null object
pH Classification    63 non-null object
                     63 non-null object
Meas                 63 non-null object
Auto Start Time      63 non-null object
MII Reflux           63 non-null object
pH Classification    63 non-null object
                     63 non-null object
dtypes: object(10)
memory usage: 5.4+ KB


# Test 3: Concatenate vertically - CPG, Auto - Patient 1

In [13]:
CPG1b = pd.read_csv('Barretts_Analysis/CPG/001 CPG.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = CPG1b.iloc[0]
header = header.str.replace('\n', ' ')
CPG1b = CPG1b.rename(columns = header)
CPG1b = CPG1b[1:]

CPG1b = CPG1b.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

CPG1b = CPG1b[CPG1b["Meas Type"].str.contains("MII")]
CPG1b['Doc'] = 'CPG'

Auto1b = pd.read_csv('Barretts_Analysis/Autoscan/001 Autoscan.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = Auto1b.iloc[0]
header = header.str.replace('\n', ' ')
Auto1b = Auto1b.rename(columns = header)
Auto1b = Auto1b[1:]

Auto1b = Auto1b.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

Auto1b = Auto1b[Auto1b["Meas Type"].str.contains("MII")]
Auto1b['Doc'] = 'Auto'

In [14]:
CPG1b

,Meas,Meas Start Time,Meas Type,Body Position,Contents,MII Reflux,pH Classification,,Doc
1,Meas 1:,1/16:42:24.0,MII,Upright,Mixed,Reflux,Acid,,CPG
12,Meas 4:,1/16:46:55.6,MII,Upright,Mixed,Reflux,Acid,,CPG
21,Meas 6:,1/16:57:22.0,MII,Upright,Mixed,Reflux,Acid,,CPG
30,Meas 8:,1/18:08:27.0,MII,Upright,Mixed,Reflux,NonAcid,,CPG
36,Meas 9:,1/18:32:44.0,MII,Upright,Mixed,Reflux,Acid,,CPG
46,Meas 11:,1/18:41:27.5,MII,Upright,Mixed,Reflux,Acid,,CPG
56,Meas 13:,1/18:53:01.0,MII,Upright,Mixed,Reflux,Acid,,CPG
65,Meas 15:,1/19:04:10.0,MII,Upright,Mixed,Reflux,Acid,,CPG
74,Meas 17:,1/19:05:54.1,MII,Upright,Mixed,Reflux,Acid,,CPG
84,Meas 19:,1/19:06:21.7,MII,Upright,Mixed,Reflux,Acid,,CPG


In [15]:
TestCPGAuto3 = pd.concat([CPG1b, Auto1b]).fillna('').drop(columns = {'Meas Type', 'Body Position', 'Contents'})

In [16]:
TestCPGAuto3

,Meas,Meas Start Time,MII Reflux,pH Classification,,Doc
1,Meas 1:,1/16:42:24.0,Reflux,Acid,,CPG
12,Meas 4:,1/16:46:55.6,Reflux,Acid,,CPG
21,Meas 6:,1/16:57:22.0,Reflux,Acid,,CPG
30,Meas 8:,1/18:08:27.0,Reflux,NonAcid,,CPG
36,Meas 9:,1/18:32:44.0,Reflux,Acid,,CPG
...,...,...,...,...,...,...
329,Meas 80:,2/11:57:13.0,Reflux,Acid,,Auto
341,Meas 83:,2/12:08:02.6,No Reflux,Needs pH,,Auto
349,Meas 86:,2/12:18:02.0,Reflux,Acid,,Auto
365,Meas 91:,2/12:32:51.0,Reflux,Acid,,Auto


In [17]:
TestCPGAuto3.Doc.value_counts()

Auto    39
CPG     37
Name: Doc, dtype: int64

Remember: 
    # If you have a Doc column again and stack vertically, you could sort by start time. 
    # Probably need Start Day and Start Time re.split

In [18]:
pd.set_option('display.max_rows', 76)

In [19]:
# df.sort_values(by='Date')

TestCPGAuto3.sort_values(by = 'Meas Start Time').reset_index().rename(columns = {'index': 'Old Index'})

,Old Index,Meas,Meas Start Time,MII Reflux,pH Classification,,Doc
0,1,Meas 1:,1/16:35:31.0,Reflux,NonAcid,,Auto
1,7,Meas 2:,1/16:42:24.0,Reflux,Acid,,Auto
2,1,Meas 1:,1/16:42:24.0,Reflux,Acid,,CPG
3,12,Meas 4:,1/16:46:55.6,Reflux,Acid,,CPG
4,18,Meas 5:,1/16:47:10.0,Reflux,Acid,,Auto
5,26,Meas 7:,1/16:57:22.0,Reflux,Acid,,Auto
6,21,Meas 6:,1/16:57:22.0,Reflux,Acid,,CPG
7,35,Meas 9:,1/18:08:27.0,Reflux,NonAcid,,Auto
8,30,Meas 8:,1/18:08:27.0,Reflux,NonAcid,,CPG
9,41,Meas 10:,1/18:32:44.0,Reflux,Acid,,Auto


* Could I write a function to read in the files and label them with the patient number? And Doc name? 

* Do I still need to split the start time? Should it be a datetime object? 

* Could you do ML to find out who is predicting better? Reflux = 1, No Reflux = 0

# Try to bring in all data & label accordingly 

In [37]:
import pandas as pd
import glob

path = 'Barretts_Analysis/'
next_path = glob.glob(path + "*")
#all_files = glob.glob(next_path + "/*.csv")



# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# command + ? = BIG comment

li = []

for some_path in next_path: 
    #print(some_path) 
    if some_path == 'Barretts_Analysis/AI': 
        continue
    else: 
        for file in glob.glob(some_path + "/*.csv"):
            #print(file)
            df = pd.read_csv(file, index_col=None, header=0, skiprows = 7).fillna('')
            header = df.iloc[0]
            header = header.str.replace('\n', ' ')
            df = df.rename(columns = header)
            df = df[1:]
            df = df.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])
            df = df[df["Meas Type"].str.contains("MII")]
            #print(df.head())
            li.append(df)
        
frame = pd.concat(li, axis=0, ignore_index=True)
# command + ? = BIG comment

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [38]:
frame.head()

,,Body Position,Contents,MII Reflux,Meas,Meas Start Time,Meas Type,pH Classification
0,,Upright,Liquid,Reflux,Meas 1:,1/6:58:52.0,MII,NonAcid
1,,Upright,Mixed,Reflux,Meas 2:,1/7:17:45.0,MII,Acid
2,,Upright,Mixed,Reflux,Meas 3:,1/7:44:58.0,MII,NonAcid
3,,Upright,Mixed,Reflux,Meas 4:,1/7:46:44.0,MII,NonAcid
4,,Upright,Mixed,Reflux,Meas 5:,1/7:48:10.0,MII,NonAcid


In [40]:
frame = frame.drop(columns = {'Meas Type', 'Body Position', 'Contents'})

In [42]:
frame.head()

,,MII Reflux,Meas,Meas Start Time,pH Classification
0,,Reflux,Meas 1:,1/6:58:52.0,NonAcid
1,,Reflux,Meas 2:,1/7:17:45.0,Acid
2,,Reflux,Meas 3:,1/7:44:58.0,NonAcid
3,,Reflux,Meas 4:,1/7:46:44.0,NonAcid
4,,Reflux,Meas 5:,1/7:48:10.0,NonAcid


But I don't have my doc name or patient number....!